## GCS Demonstration and Development Benchmark

(cc) conrad.jackisch@tbt.tu-freiberg.de, antita.sanchez@mineral.tu-freiberg.de  

This notebook demonstrates the integrated hysteresis analysis package GCS combining four methods:
1. **HARP** (Roberts et al., 2023) - Empirical classification based on peak timing and loop geometry
2. **Zuecco** (Zuecco et al., 2016) - Integration-based index with 9-class classification
3. **Lloyd/Lawler** (Lloyd et al., 2016; Lawler et al., 2006) - Percentile-based indices
4. **Musolff/Thompson** (Musolff et al., 2015; Thompson et al., 2011) - ratio of CV of C and Q 

It is intended as a functionality benchmark using the same reference as the Zuecco package https://github.com/florianjehn/Hysteresis-Index-Zuecco and HARP package https://github.com/MelanieEmmajade/HARP.
It shall enable a critical usage of the C-Q analysis in the sense of Knapp and Musolff, 2024 https://doi.org/10.1002/hyp.15328.

## HOW THE THREE METHODS COMPLEMENT EACH OTHER

### HARP METHOD
 - Provides qualitative empirical classifica
 - Identifies process timing (flushing vs. dilution)
 - Easy interpretation with named classes
 - Best for: First-pass analysis and process identification

### ZUECCO METHOD
 - Quantifies loop area and hysteresis strength
 - 9-class system captures subtle variations
 - Detects mixed clockwise/counter-clockwise patterns
 - Best for: Quantitative comparisons and complex patterns

### LLOYD/LAWLER METHODS
 - Percentile-based for standard comparisons
 - HInew has symmetric range [-1, 1]
 - Shows flow-dependent hysteresis
 - Best for: Publication-quality quantification

### MUSLOFF/THOMPSON METHODS
 - Analysis in ∆C/∆Q and statistical spaces of their covariates

**✓ CONVERGENT EVIDENCE = HIGH CONFIDENCE**
When all three methods agree → Strong, unambiguous pattern

**⚠️ DIVERGENT RESULTS = INVESTIGATE FURTHER**
When methods disagree → Complex/mixed pattern

## GEOCHEMICAL CLASSIFICATION AND RESULT VISUALIZATION

Above the standard classifications from the individual methods and an evaluation about their coherence, the package contains a percentile and C/Q analysis based classification approach, which is developed for Sanchez et al. 2025 (in review). 

To evaluate its basis and to deeply dive into the actual structure of the C-Q dynamics, we provide several comprehensive plotting options. 

## DISCLAIMER

Despite all testing and care, this code is scientific and experimental. Do not trust the results before throughout analysis. The code has a couple of more or less hard-coded assumptions (like usual time series in days and spline interpolations) which might completely skew up under untested circumstances. Feel free to fork, contribute, extend with cc-by.

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sys
sys.path.append('../hygcs')
# Import the integrated hysteresis system (v4 - for event-scale analysis)
import gcs as gcs

# Import individual methods and plotting functions
from harp import calculate_harp_metrics, harp_plot
from zuecco import calculate_zuecco_metrics, zuecco_plot
from lloyd import calculate_lawlerlloyd_metrics, lloyd_plot

# Note: For time series classification, we'll use gcs_v5 (imported in Section 6)

print("✓ All modules loaded successfully")

In [ ]:
# Load example datasets
sheet_names = ["soil_1", "soil_2", "connectivity_1", "connectivity_2"]

datasets = {}
for idx, name in enumerate(sheet_names):
    df = pd.read_excel(
        "hysteresis_examples.xlsx",
        sheet_name=idx,
        index_col=0
    ).reset_index()
    datasets[name] = df
    print(f"Loaded {name}: {len(df)} points, columns: {list(df.columns)}")

## 2. Single Event Analysis - Complete Workflow

We'll analyze the `soil_2` dataset in detail, demonstrating all three methods.

In [ ]:
# Select dataset for detailed analysis
test_data = datasets['soil_2'].copy()

# Extract column names
time_col = test_data.columns[0]
discharge_col = test_data.columns[1]
concentration_col = test_data.columns[2]

print(f"Analyzing: soil_2 dataset")
print(f"  Time: {time_col}")
print(f"  Discharge: {discharge_col}")
print(f"  Concentration: {concentration_col}")
print(f"  Data points: {len(test_data)}")
print(f"  Time range: {test_data[time_col].min()} to {test_data[time_col].max()}")

In [ ]:
#test_data

### 2.1 Calculate All Hysteresis Metrics

The `calculate_all_hysteresis_metrics()` function is our comprehensive wrapper that applies all three methods to the event data.

In [ ]:
# Calculate all metrics
all_metrics = gcs.calculate_all_hysteresis_metrics(
    test_data,
    time_col=time_col,
    discharge_col=discharge_col,
    concentration_col=concentration_col
)

# Check for errors
if 'error' in all_metrics and all_metrics['error'] != 'None':
    print(f"⚠️ Error occurred: {all_metrics['error']}")
else:
    print("✓ All metrics calculated successfully\n")

### 2.2 Calculate the individual metrics for direct testing

In [ ]:
harp_metrics, harp_df = gcs.calculate_harp_metrics(test_data, time_col=time_col, discharge_col=discharge_col, concentration_col=concentration_col)
harp_metrics

In [ ]:
zuecco_metrics, zuecco_df = gcs.calculate_zuecco_metrics(test_data, time_col=time_col, discharge_col=discharge_col, concentration_col=concentration_col)
zuecco_metrics

In [ ]:
lloyd_metrics, lloyd_df = gcs.calculate_lawlerlloyd_metrics(test_data, time_col=time_col, discharge_col=discharge_col, concentration_col=concentration_col)
lloyd_metrics

In [ ]:
#all_metrics

### 2.3.1 HARP Method Results

**HARP (Hysteresis Analysis of Rising and Falling Peaks)** uses empirical classification based on:
- Peak timing difference between Q and C
- Loop shape and area
- Residual (end-state deviation)

In [ ]:
harp_metrics = all_metrics['harp_metrics']

print("═" * 60)
print("HARP METHOD RESULTS")
print("═" * 60)

if harp_metrics:
    print(f"\nKey Metrics:")
    print(f"  • Loop Area: {harp_metrics.get('area', np.nan):.4f}")
    print(f"  • Residual: {harp_metrics.get('residual', np.nan):.4f}")
    print(f"  • Peak Q timing: {harp_metrics.get('peaktime_Q', np.nan):.4f} days")
    print(f"  • Peak C timing: {harp_metrics.get('peaktime_C', np.nan):.4f} days")
    
    # Interpret peak timing
    peak_q = harp_metrics.get('peaktime_Q', np.nan)
    peak_c = harp_metrics.get('peaktime_C', np.nan)
    if not np.isnan(peak_q) and not np.isnan(peak_c):
        diff = peak_c - peak_q
        if diff < 0:
            timing_interp = f"C peaks {abs(diff):.2f} days BEFORE Q → Flushing/mobilization"
        elif diff > 0:
            timing_interp = f"C peaks {diff:.2f} days AFTER Q → Dilution/depletion"
        else:
            timing_interp = "Peaks coincide → Complex/mixed"
        print(f"\nInterpretation: {timing_interp}")
else:
    print("No HARP metrics available")

### 2.3.2 Zuecco Method Results

**Zuecco Index** calculates the integrated area between rising and falling limbs:
- h_index: Sum of differential areas (can be positive or negative)
- 9-class classification system (0-8)
- Classes 1-4: Clockwise variants
- Classes 5-8: Counter-clockwise variants
- Class 0: Linear/no hysteresis

In [ ]:
zuecco_metrics = all_metrics['zuecco_metrics']

print("═" * 60)
print("ZUECCO METHOD RESULTS")
print("═" * 60)

if zuecco_metrics:
    h_index = zuecco_metrics.get('h_index', np.nan)
    hyst_class = zuecco_metrics.get('hyst_class', -1)
    
    print(f"\nClassification: Class {hyst_class}")
    print(f"\nKey Metrics:")
    print(f"  • h-index: {h_index:.4f}")
    print(f"  • Min differential area: {zuecco_metrics.get('min_diff_area', np.nan):.6f}")
    print(f"  • Max differential area: {zuecco_metrics.get('max_diff_area', np.nan):.6f}")
    
    # Interpret h-index magnitude
    if not np.isnan(h_index):
        if h_index > 0.1:
            mag_interp = "Strong clockwise hysteresis"
        elif h_index > 0.01:
            mag_interp = "Moderate clockwise hysteresis"
        elif h_index > -0.01:
            mag_interp = "Weak/no hysteresis"
        elif h_index > -0.1:
            mag_interp = "Moderate counter-clockwise hysteresis"
        else:
            mag_interp = "Strong counter-clockwise hysteresis"
        print(f"\nInterpretation: {mag_interp}")
else:
    print("No Zuecco metrics available")

### 2.3.3 Lloyd/Lawler Method Results

**Two complementary percentile-based methods:**

1. **Lloyd et al. (2016) - HInew (RECOMMENDED)**
   - Difference-based: `HI = (C_rise - C_fall) / C_mid`
   - Symmetric range: [-1, 1]
   - Better comparability

2. **Lawler et al. (2006) - HIL (ORIGINAL)**
   - Ratio-based: `HI = (C_rise / C_fall) - 1` or `(-1 / ratio) + 1`
   - Asymmetric range
   - More sensitive at extremes

Both sample at 9 discharge percentiles (0.1 to 0.9)

In [ ]:
lloyd_metrics = all_metrics['lloyd_metrics']
classifications = all_metrics['classifications']

print("═" * 60)
print("LLOYD/LAWLER METHOD RESULTS")
print("═" * 60)

if lloyd_metrics:
    # Lloyd (2016) - Recommended method
    print("\nLLOYD (2016) - DIFFERENCE METHOD [RECOMMENDED]")
    print(f"   Direction: {classifications.get('lloyd_direction', 'unknown')}")
    print(f"\n   Key Metrics:")
    print(f"     • Mean HInew: {lloyd_metrics.get('mean_HInew', np.nan):.4f}")
    print(f"     • Median HInew: {lloyd_metrics.get('median_HInew', np.nan):.4f}")
    print(f"     • HInew Range: {lloyd_metrics.get('HInew_range', np.nan):.4f}")
    
    # Lawler (2006) - Original method
    print("\nLAWLER (2006) - RATIO METHOD [COMPARISON]")
    print(f"   Direction: {classifications.get('lawler_direction', 'unknown')}")
    print(f"\n   Key Metrics:")
    print(f"     • Mean HIL: {lloyd_metrics.get('mean_HIL', np.nan):.4f}")
    print(f"     • Median HIL: {lloyd_metrics.get('median_HIL', np.nan):.4f}")
    
    # Magnitude interpretation
    mean_hinew = lloyd_metrics.get('mean_HInew', np.nan)
    if not np.isnan(mean_hinew):
        if abs(mean_hinew) > 0.5:
            strength = "STRONG"
        elif abs(mean_hinew) > 0.2:
            strength = "MODERATE"
        elif abs(mean_hinew) > 0.05:
            strength = "WEAK"
        else:
            strength = "NEGLIGIBLE"
        print(f"\nInterpretation: {strength} hysteresis (|HInew| = {abs(mean_hinew):.3f})")
else:
    print("No Lloyd/Lawler metrics available")

## 3. Visual Analysis - All Three Methods

Now let's visualize the results from each method. We'll call the individual calculation functions to get the complete DataFrames with attrs needed for plotting.

In [ ]:
# Calculate metrics using individual functions to get full DataFrames with attrs
harp_metrics_df, harp_df = gcs.calculate_harp_metrics(
    test_data, time_col=time_col, 
    discharge_col=discharge_col, 
    concentration_col=concentration_col
)

zuecco_metrics_df, zuecco_df = gcs.calculate_zuecco_metrics(
    test_data, time_col=time_col, 
    discharge_col=discharge_col, 
    concentration_col=concentration_col
)

lloyd_metrics_df, lloyd_df = gcs.calculate_lawlerlloyd_metrics(
    test_data, time_col=time_col, 
    discharge_col=discharge_col, 
    concentration_col=concentration_col
)

print("✓ Individual metrics calculated for plotting")

In [ ]:
fig_harp = gcs.harp_plot(harp_df, harp_metrics_df)
fig_harp.update_layout(title="HARP Analysis - soil_2 Dataset")
fig_harp.show()

### 3.2 Zuecco Visualization

In [ ]:
fig_zuecco = gcs.zuecco_plot(zuecco_df, zuecco_metrics_df)
fig_zuecco.update_layout(title="Zuecco Analysis - soil_2 Dataset")
fig_zuecco.show()

### 3.3 Lloyd/Lawler Visualization

In [ ]:
fig_lloyd = gcs.lloyd_plot(lloyd_df, lloyd_metrics_df)
fig_lloyd.update_layout(title="Lloyd/Lawler Analysis - soil_2 Dataset")
fig_lloyd.show()

## 4. Comparative Analysis - All Four Datasets

Let's analyze all four example datasets to compare hysteresis patterns.

In [ ]:
# Analyze all datasets
results_summary = []

for name, df in datasets.items():
    time_col = df.columns[0]
    discharge_col = df.columns[1]
    concentration_col = df.columns[2]
    
    metrics = gcs.calculate_all_hysteresis_metrics(
        df, time_col=time_col, 
        discharge_col=discharge_col, 
        concentration_col=concentration_col
    )
    
    # Extract key metrics
    harp_m = metrics['harp_metrics']
    zuecco_m = metrics['zuecco_metrics']
    lloyd_m = metrics['lloyd_metrics']
    class_m = metrics['classifications']
    
    results_summary.append({
        'Dataset': name,
        'N_points': len(df),
        'HARP_area': harp_m.get('area', np.nan),
        'Zuecco_h_index': zuecco_m.get('h_index', np.nan),
        'Zuecco_class': zuecco_m.get('hyst_class', -1),
        'Lloyd_mean_HInew': lloyd_m.get('mean_HInew', np.nan),
        'Lloyd_direction': class_m.get('lloyd_direction', 'unknown'),
        'Lawler_mean_HIL': lloyd_m.get('mean_HIL', np.nan)
    })

summary_df = pd.DataFrame(results_summary)
print("\n" + "═" * 100)
print("COMPARATIVE SUMMARY - ALL DATASETS")
print("═" * 100)
display(summary_df)

In [ ]:
# Create visual comparison
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        f"{name}<br>Lloyd HI={summary_df.loc[i, 'Lloyd_mean_HInew']:.3f}, Zuecco={summary_df.loc[i, 'Zuecco_class']}"
        for i, name in enumerate(sheet_names)
    ],
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
           [{'type': 'scatter'}, {'type': 'scatter'}]]
)

for idx, (name, df) in enumerate(datasets.items()):
    row = idx // 2 + 1
    col = idx % 2 + 1
    
    # Normalize data for plotting
    Q = df.iloc[:, 1].values
    C = df.iloc[:, 2].values
    Q_norm = (Q - Q.min()) / (Q.max() - Q.min()) if Q.max() > Q.min() else Q
    C_norm = (C - C.min()) / (C.max() - C.min()) if C.max() > C.min() else C
    
    fig.add_trace(
        go.Scatter(
            x=Q_norm, y=C_norm,
            mode='lines+markers',
            marker=dict(size=4, color=np.arange(len(Q_norm)), colorscale='Viridis'),
            line=dict(color='gray', width=1),
            name=name,
            showlegend=False
        ),
        row=row, col=col
    )
    
    fig.update_xaxes(title_text="Q (normalized)", row=row, col=col)
    fig.update_yaxes(title_text="C (normalized)", row=row, col=col)

fig.update_layout(
    title="Hysteresis Loops - All Datasets (colored by time)",
    height=700,
    template='none'
)

fig.show()

## 5. Overall Conclusions

**For Event-Scale Hysteresis Analysis:**
- **Use all three methods** for robust analysis
- **Lloyd HInew** recommended for standard reporting
- **Zuecco** excellent for complex/mixed patterns
- **HARP** intuitive for process identification
- **Convergent evidence** = high confidence

**For Time Series Classification (GCS v5.0):**
- **Integrates hysteresis + C-Q dynamics** for mechanistic phase classification
- **C-Q slopes** reveal connectivity patterns (dilution vs enrichment)
- **Window-scale hysteresis** captures temporal dynamics correctly
- **Percentile-based** thresholds work across compounds
- **6 phases** (F, L, C, D, R, V) capture geochemical behavior

### Differences Between Approaches:

| Aspect | Event Hysteresis | GCS Time Series |
|--------|------------------|-----------------|
| **Scope** | Single event (storm, flush) | Multiple cycles over time |
| **Input** | One C-Q loop | Time series with many points |
| **Output** | HI, class, metrics | Phase per segment + confidence |
| **Hysteresis** | Event-scale | Window-scale per segment |
| **Best for** | Understanding single events | Long-term monitoring analysis |

The **Hydro-Geochemical Classification Suite (HyGCS)** integrates hysteresis analysis with C-Q dynamics to classify time series data into geochemical phases:

- **F (Flushing)**: Dilution-dominated, steep C decline during high Q
- **L (Loading)**: Enrichment before peak, C increase with Q
- **C (Chemostatic)**: Buffered, low CVc/CVq, flat C-Q slope
- **D (Dilution)**: Post-flush recovery, depleted sources
- **R (Recession)**: Late cycle, low variability, connectivity loss
- **V (Variable)**: Mixed/ambiguous patterns

**Note:** HyGCS expects **time series data** with multiple sampling points, whereas the HARP/Lloyd/Zuecco examples are single-event hysteresis loops. We'll demonstrate the classifier conceptually using a shortened time series. Please head to `test_gcs.ipynb` to continue.